In [ ]:
import thinkdsp
from thinkdsp import CosSignal, SinSignal
from thinkdsp import decorate
from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt
import os
from itertools import combinations
import seaborn as sns

In [ ]:
# How we get pseudo-Harmony Function
def blur_minfac(a,b, max_limit = 10000, delta = 0.01): #Take the quotient of the least common multiple of a and b divided by the smaller number
    if a > b:
        c = a
        a = b
        b = c      
    times = 1
    while True:
        if b*times%a <= times*delta*a  or times > max_limit:
            break
        else:
            #print(times)
            times += 1
    
    return b*times//a

x = np.arange(1,2.025,0.025)
delta = 0.02

print(f'Delta = {delta}')
y = []
for i in x:
    i = round(i,3)
    y.append(blur_minfac(1,i,delta = delta))

y = np.array(y)
y = 1/y

log_x = np.log2(x)
plt.figure(figsize=(10, 6), dpi=400, facecolor='#FFFFFF', edgecolor='#0000FF')
plt.scatter(log_x,y,s = 20)
plt.plot(log_x,y)

xs = np.arange(0,1+1/12,1/12)

ys = [0,1]
for i in xs:
    a = i*np.ones(len(ys))
    plt.plot(a,ys, 'k--')

    
labels = ['C','#C','D','#D','E','F','#F','G','#G','A','#A','B','C']
plt.xticks(ticks = xs, labels = labels)

In [ ]:
#Hrmony Fuction
harmonic = 'Full'
if harmonic == "Full":
    a = []
    b = []

    ax = np.arange(1,2.025,0.025)
    log_x = np.log2(ax)
    bx = [1.0, 0.024390243902439025, 0.047619047619047616, 0.06666666666666667, 0.09090909090909091, 0.1111111111111111, 0.125, 0.14285714285714285, 0.16666666666666666, 0.09090909090909091, 0.2, 0.07142857142857142, 0.1111111111111111, 0.058823529411764705, 0.25, 0.09090909090909091, 0.14285714285714285, 0.058823529411764705, 0.07692307692307693, 0.05263157894736842, 0.3333333333333333, 0.03125, 0.058823529411764705, 0.09090909090909091, 0.125, 0.07692307692307693, 0.05555555555555555, 0.2, 0.058823529411764705, 0.08333333333333333, 0.14285714285714285, 0.043478260869565216, 0.1111111111111111, 0.05, 0.09090909090909091, 0.07692307692307693, 0.058823529411764705, 0.047619047619047616, 0.034482758620689655, 0.022222222222222223, 0.5]

    ax_reverse = ax[1:][::-1]
    ax_reverse = np.array(ax_reverse)
    ax_reverse = 1/ax_reverse

    bx_reverse = bx[1:][::-1]

    for i in range(len(ax_reverse)):
        b.append(bx_reverse[i])
        a.append(ax_reverse[i])

    for i in range(len(ax)):
        a.append(ax[i])
        b.append(bx[i])

    ax = np.array(a)
    #plt.plot(np.log2(ax),b)

    ax = np.array(a)
    bx = np.array(b)
    #bx = bx**2
    #plt.savefig('fig4.png')
    
elif harmonic == "Ease":
    ax = np.array([0.9756097560975611, 1.0, 1.025])
    bx = np.array([0.024390243902439025, 1.0, 0.024390243902439025])

In [ ]:
#Comparison Fuction

def fa(freq, hs, fs):
    hs = abs(hs)
    result = np.interp(fs, freq * ax, bx, left = 0,right = 0) * hs
    return result.sum()

def compare(spec1, spec2, threshold = 500 ,top_k = 10,features = 99999, stand = 'sum', square = False, root = False):
    fs1 = spec1.fs[10:]
    fs2 = spec2.fs[10:]

    hs1 = spec1.hs[10:]
    hs2 = spec2.hs[10:]
    
    hs1 = abs(hs1)
    hs2 = abs(hs2)

    #removing frequencies whose initial amplitude is below the threshold 
    hs1[hs1<threshold] = 0
    hs2[hs2<threshold] = 0
    
    #setting the amplitude of the frequency with the largest amplitude to 1
    if stand == 'max':
        hs1 /= hs1.max()
        hs2 /= hs2.max()
    elif stand == 'sum':
        hs1 /= hs1.sum()
        hs2 /= hs2.sum()
    
    if square == True: 
        hs1 = hs1**2
        hs2 = hs2**2
        
    #Base Freq
    top_k= top_k
    max1 = hs1.argsort()[::-1][0:top_k].min()
    max2 = hs2.argsort()[::-1][0:top_k].min()
    
    #Align Base Freq
    if max1 >= max2:
        fs1 = np.log2(fs1)
        fs1 -= fs1[max1] - np.log2(fs2[max2])
        fs1 = 2**(fs1)
        
    else:
        fs2 = np.log2(fs2)
        fs2 -= fs2[max2] - np.log2(fs1[max1])
        fs2 = 2**(fs2)
    
    spec1.hs = hs1
    spec2.hs = hs2
    spec1.fs = fs1
    spec2.fs = fs2
    
    spec1.plot(high = 500)
    spec2.plot(high = 500)
    #plt.show()

    if features != 99999:
        max1 = hs1.argsort()[::-1][0:features]
        fs1 = fs1[max1]
        hs1 = hs1[max1]

        max2 = hs2.argsort()[::-1][0:features]
        fs2 = fs2[max2]
        hs2 = hs2[max2]
    
    tmp_sum = 0
    
    #Sum up
    for i in range(len(fs1)):
        if hs1[i] != 0:
            tmp_sum += fa(fs1[i],hs2,fs2)
        else:
            continue
   
    if root == True:
        tmp_sum /= (np.sum(hs1>0)*np.sum(hs2>0))**0.5
        
    return tmp_sum

In [ ]:
#Samples decision

samples = os.listdir('Sampless')
samples.sort()
samples = samples[1:]
for i in range(len(samples)):
    samples[i] = 'Sampless/'+samples[i]

sample_type = '6guitar+6piano+9strings'
if sample_type == '6guitar+6piano':
    samples = samples[:12]
    split = [3,6,9]
elif sample_type == '6guitar+6piano+9synth':
    samples = samples[:21]
    split = [3,6,9,12,15,18]
elif sample_type == '9synth':
    samples =  samples[12:21]
    split = [3,6]
elif sample_type == '6guitar+6piano+9strings':
    samples = samples[:12] + samples[21:30]
    split = [3,6,9,12,15,18]

for u in samples:
    print(u)

In [ ]:
final_array = np.ones([len(samples),len(samples)])

threshold = 500
top_k = 10
features = 99999
stand = 'sum'
square = False
root = False

for i in range(len(samples)):
    wave = thinkdsp.read_wave(samples[i])
    spec = wave.make_spectrum()
    tmp = compare(spec,spec,threshold,top_k,features, stand, square, root)
    
    final_array[i][i] = tmp
    
    print(i,':',tmp, end = ' || ')
    
s = time.time()

for i in combinations(samples,2):
    wave1 = thinkdsp.read_wave(i[0])
    spec1 = wave1.make_spectrum()

    wave2 = thinkdsp.read_wave(i[1])
    spec2 = wave2.make_spectrum()
    
    var = compare(spec1,spec2,threshold,top_k,features, stand, square, root) / (final_array[samples.index(i[1])][samples.index(i[1])] * final_array[samples.index(i[0])][samples.index(i[0])])**0.5
    
    final_array[samples.index(i[0])][samples.index(i[1])] = var
    final_array[samples.index(i[1])][samples.index(i[0])] = var

print('Done',delta/compares)

In [ ]:
plt.figure(figsize=(10, 6), dpi=400)


for i in range(len(final_array)):
    final_array[i][i] /= final_array[i][i]
    
log = False

if log == True:
    fig = sns.heatmap(np.log10(final_array), cmap = 'Greys')
else:
    fig = sns.heatmap((final_array), cmap = 'Greys')
fig.hlines(split,*fig.get_xlim(),color = 'black')
fig.vlines(split,*fig.get_ylim(),color = 'black')

print(f'{log}{harmonic}{sample_type}thre={threshold}top_k={top_k}fea={features}stand={stand}square={square}root={root}')
plt.savefig(f'tmp/{log}{harmonic}{sample_type}thre={threshold}top_k={top_k}fea={features}stand={stand}square={square}root={root}.png')
plt.show()